In [1]:
import sys

sys.path.append('../')

In [2]:
from qiskit.providers.fake_provider import FakeMontreal
from qiskit import IBMQ
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer import QasmSimulator

from molecules import compute_molecule
from measurements import compute_energy, create_circuits
from grouping import Grouping
from utils import extract_paulis, send_ibmq_parallel

/home/dfernandez/anaconda3/envs/HEEM/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dfernandez/anaconda3/envs/HEEM/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/dfernandez/anaconda3/envs/HEEM/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dfernandez/anaconda3/envs/HEEM/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [4]:
IBMQ.load_account()
provider_main = IBMQ.get_provider()
backend_IBMQ = provider_main.get_backend('ibmq_qasm_simulator')

NUM_SHOTS = 2 ** 14

backend = FakeMontreal()
WC_device = backend.configuration().coupling_map
WC_device = [tuple(x) for x in WC_device]

device = QasmSimulator.from_backend(backend)
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device)

In [12]:
molecule_name = 'CH4'
qubit_op = compute_molecule(molecule_name)

In [13]:
labels, coeffs = extract_paulis(qubit_op)
energy_exact = 0

for i in range(len(labels)):
    label = labels[i]
    if 'X' not in label and 'Y' not in label:
        energy_exact += coeffs[i].real

print('Exact energy: {}'.format(energy_exact))

Exact energy: -10.66250632013532


In [14]:
grouping_HEEM = Grouping(qubit_op, connectivity=WC_device, print_progress=True, method='HEEM', connected_graph=True)
grouping_HEEM.group()
circuits = create_circuits(grouping_HEEM.measurements, qubit_op.num_qubits)

print(f'A total of {len(circuits)} circuits will be simulated')

Pauli graph:   0%|          | 0/1039 [00:00<?, ?it/s]

Grouping entangled:   0%|          | 0/1039 [00:00<?, ?it/s]

A total of 129 circuits will be simulated


In [15]:
jobs = send_ibmq_parallel(backend_IBMQ, circuits, job_tags=[molecule_name, 'HEEM'], progress_bar=True,
                          circuits_batch_size=33, shots=NUM_SHOTS, coupling_map=coupling_map,
                          initial_layout=grouping_HEEM.T, noise_model=noise_model)

Jobs completed:   0%|          | 0/4 [00:00<?, ?it/s]

In [16]:
counts = []

for job in jobs:
    temp = job.result().get_counts()
    if type(temp) is list:
        counts += temp
    else:
        counts.append(temp)

In [17]:
energy_simulation = compute_energy(counts, labels, coeffs, grouping_HEEM.groups, grouping_HEEM.measurements, NUM_SHOTS)
print('Simulated energy: {}'.format(energy_simulation))

Computing energy:   0%|          | 0/129 [00:00<?, ?it/s]

Simulated energy: -11.727498116660529
